In [ ]:
import torch
from torch import nn
import torchvision
!wget -O helper_functions.py https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py
from helper_functions import accuracy_fn
from torchvision import datasets
from torchvision.transforms import ToTensor
train_data=datasets.MNIST(root="data",train=True,download=True,transform=ToTensor(),target_transform=None)
test_data=datasets.MNIST(root="data",train=False,download=True,transform=ToTensor())
from torch.utils.data import DataLoader
size=32
train_dataloader=DataLoader(train_data,batch_size=size,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=size,shuffle=False)
from torch import nn
class_names=train_data.classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,y_pred=y_pred.argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            test_pred = model(X)
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1))
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")
class FashionMNISTModel1(nn.Module):
  def __init__(self,input_shape: int,hidden_units: int, output_shape: int):
    super().__init__()
    self.block1=nn.Sequential(
        nn.Conv2d(in_channels=input_shape,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.block2=nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7,out_features=output_shape)
        )
  def forward(self,x):
    x=self.block1(x)
    x=self.block2(x)
    x=self.classifier(x)
    return x
torch.manual_seed(42)
model2=FashionMNISTModel1(input_shape=1,hidden_units=10,output_shape=len(class_names)).to(device)
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model2.parameters(),lr=0.1)
torch.manual_seed(42)
epochs=3
for epoch in range(epochs):
  print(f"Epoch: {epoch}\n-----")
  train_step(data_loader=train_dataloader,
             model=model2,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn,
             device=device)
  test_step(data_loader=test_dataloader,
        model=model2,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        device=device
    )





In [ ]:
import torch
import torchvision.models import models
models=list_models()